# Download images from XNAT

In [1]:
import os
path_to_repository = '/home/grg/git/xnat-monitor'
os.chdir(path_to_repository)
import bbrc

Setting up a connection to XNAT and defining which images to retrieve.

In [13]:
# Author: Grégory Operto <goperto@barcelonabeta.org>

from pyxnat import Interface
from bbrc import xnat

config_file = '/home/grg/.xnat_bsc.cfg'
dest_dir = '/tmp/amyloid2'
project = 'ALFA_OPCIONAL'
subjects = ['11540'] #[e for e in open('/tmp/amyloid_subjects.txt').read().split('\n') if e != '']
sequences = ['T1_ALFA1', 'DWI_ALFA1']

# Setting up an active connexion to XNAT
central = Interface(config=config_file)
print subjects, len(subjects)

['11540'] 1


In [14]:
# Selecting a project from which download images
p = central.select.project(project)
# Get a list of subjects in the project to download all of them (instead of a list)
#subjects = [e.label() for e in list(p.subjects())]

for subject in subjects:
    # For each subject iterate on associated experiments
    exp = list(p.subject(str(subject)).experiments())    
    for e in exp:
        # Get a list of available sequences
        d = xnat.series_desc(central, e.id())        
        print('%s has the following sequences %s'%(subject, d))
        for sq in sequences:                    
            # Get the scan corresponding to sequence if available and select the NIFTI resource
            if sq in d.values():
                sids = [k for k,v in d.items() if v == sq and not k.startswith('0')]
                assert(len(sids)!=0)
                for sid in sids:
                    sc = e.scan(sid)
                    res = sc.resource('NIFTI')

                    # Displays a list of contained files
                    print('Resource files of sequence %s: %s'%(sq, list(res.files())))

                    # Download the corresponding archive and extract it in dest_dir
                    fp1 = res.get(dest_dir, extract=True)
                    print('%s have been created successfully'%fp1)
            else:
                print('%s is missing for subject %s'%(sq, subject))
    print('')

11540 has the following sequences {'201': 'B1_calibration', '601': 'RS_ALFA1', '1301': 'DWI_ALFA1', '1201': 'rDWI_ALFA1', '203': 'B1_calibration', '404': 'T1_Axial', '403': 'T1_Coronal', '401': 'T1_ALFA1', '501': 'rRS_ALFA1', '0-OT1': 'T1_Coronal', '0-OT3': 'T1_ALFA1', '0-OT2': 'T1_Axial', '0-OT5': 'DWI_ALFA1', '0-OT4': 'FLAIR_ALFA1', '301': 'B0_ALFA1', '701': 'SE-fMRI-AP', '103': 'SmartBrain SENSE', '101': 'SmartBrain SENSE', '104': 'Patient Aligned MPR AWPLAN_SMARTPLAN_TYPE_BRAIN', '903': 'FLAIR_Coronal', '901': 'FLAIR_ALFA1', '904': 'FLAIR_Axial', '1101': 'IR_ALFA1', '801': 'SE-fMRI-PA', '1001': 'T2_ALFA1'}
Resource files of sequence T1_ALFA1: [<File Object> 11540_401.nii.gz]
[u'/tmp/amyloid2/NIFTI/11540_401.nii.gz'] have been created successfully
Resource files of sequence DWI_ALFA1: [<File Object> 11540_1301.nii.gz, <File Object> 11540_1301.bvec, <File Object> 11540_1301.bval]
[u'/tmp/amyloid2/NIFTI/11540_1301.nii.gz', u'/tmp/amyloid2/NIFTI/11540_1301.bvec', u'/tmp/amyloid2/NIFTI/

Are there any missing subjects?

In [ ]:
import os.path as osp
from glob import glob
import pandas as pd

wd = '/tmp/downloads/NIFTI'
t = [[subject, len(glob(osp.join(wd, '%s*'%subject)))] for subject in subjects]
df = pd.DataFrame(t, columns=['subject', 'n'])

missing = df[df['n']==0]['subject'].tolist()
print('Missing subjects: %s'%missing)

Applying online protocol validation on first missing subject (first experiment if any)

In [ ]:
first = missing[0]
expid = xnat.subject_experiments(central, project, first)[0].id()
destination = [['goperto@barcelonabeta.org'], ['goperto@barcelonabeta.org']]
res, logs = xnat.check_session(central, expid, destination=destination, send_mail=True)
print('%s resulted with %s'%(first, ['SUCCESS', 'WARNING', 'ERRORS'][res]))